# Models for plate characters recognition

In [27]:
from sklearn import svm
import numpy as np
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn import neighbors
from sklearn import tree
#from xgboost import XGBClassifier

## General utils

In [9]:
def func_overX(X):
    Y = []
    for element in X:
        Y.append( sum(element.flatten())  > element.flatten().shape[0] //2)
    return np.asarray(Y)

def testing():

    ##### SVM Testing #####
    X = np.random.rand(200 , 60,30)
    Y = func_overX(X)
   
    ### GridSearch ###
    model = SVMCHandler(X,Y)
    model.fit(with_score=True, with_grid=True)


def _ensure_dimensionalit( arr):
        return arr if len(arr[0].shape) == 1 else [x.flatten() for x in arr]

def _acc(y_pred, y_target):

    if type(y_pred) == np.array and type(y_target) == np.array :
        assert(y_pred.shape == y_target.shape)
        mask = y_pred == y_target
    
    else: 
        assert(len(y_pred) == len(y_target))
        mask = [x==y for x,y in zip(y_pred, y_target)]
    return sum(mask)/len(mask)

#Scaler = StandardScaler()

def do_scaling(X):
    Scaler = StandardScaler()
    return Scaler.fit_transform(X)

def _gen_gridSearch(model,hyperparams):
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state= 42)
    grid = GridSearchCV(model , param_grid= hyperparams, cv=cv, n_jobs=10, verbose=3, scoring='roc_auc')
    return grid 


# Models

## SVM

In [10]:
class SVMCHandler:

    def __init__(self, X, Y, **kwargs):
        hyperparams = {
            'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
            'C': np.logspace(-2, 10 , 5),
            'gamma': np.logspace(-9,3,5),
        }
        self.model = svm.SVC(**kwargs)
        self.X = do_scaling(_ensure_dimensionalit(X))
        self.Y = Y
        assert(len(Y.shape) ==1 )
        self.grid = _gen_gridSearch(self.model, hyperparams)
        
    
    def fit(self, with_score = True, with_grid=True):
        if with_grid:
            self.grid.fit(self.X, self.Y)
            print(f"The best parameters are {self.grid.best_params_} and the best score is {self.grid.best_score_}")
            self.model = self.model.__class__(**self.grid.best_params_)
            self.model.fit(self.X,self.Y)
        else : 
            self.model.fit(self.X, self.Y )
        if with_score:
            pred = self.predict(self.X)
            print(f"Train acc  is : {_acc(pred, self.Y)}")

    def predict(self, X):
        X = _ensure_dimensionalit(X)
        return self.model.predict(X)


## DL model

## KNN model

In [14]:
class KNNhandler:

    def __init__(self, X, Y, **kwargs):
        hyperparams = {
            'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
            'n_neighbors': np.arange(3, 10, 2),
            'p': np.arange(1,3),
        }
        self.model = neighbors.KNeighborsClassifier(**kwargs)
        self.X = do_scaling(_ensure_dimensionalit(X))
        self.Y = Y
        assert(len(Y.shape) ==1 )
        self.grid = _gen_gridSearch(self.model, hyperparams)
        
    
    def fit(self, with_score = True, with_grid=True):
        if with_grid:
            self.grid.fit(self.X, self.Y)
            print(f"The best parameters are {self.grid.best_params_} and the best score is {self.grid.best_score_}")
            self.model = self.model.__class__(**self.grid.best_params_)
            self.model.fit(self.X,self.Y)
        else : 
            self.model.fit(self.X, self.Y )
        if with_score:
            pred = self.predict(self.X)
            print(f"Train acc  is : {_acc(pred, self.Y)}")

    def predict(self, X):
        X = _ensure_dimensionalit(X)
        return self.model.predict(X)


# Decision tree

In [29]:
class DTCHandler:

    def __init__(self, X, Y, **kwargs):
        hyperparams = {
            'criterion':['gini', 'entropy'],
            'max_features': ['auto', 'sqrt', 'log2'],
            'splitter': ['best', 'random'],
        }
        self.model = tree.DecisionTreeClassifier(**kwargs)
        self.X = do_scaling(_ensure_dimensionalit(X))
        self.Y = Y
        assert(len(Y.shape) ==1 )
        self.grid = _gen_gridSearch(self.model, hyperparams)
        
    
    def fit(self, with_score = True, with_grid=True):
        if with_grid:
            self.grid.fit(self.X, self.Y)
            print(f"The best parameters are {self.grid.best_params_} and the best score is {self.grid.best_score_}")
            self.model = self.model.__class__(**self.grid.best_params_)
            self.model.fit(self.X,self.Y)
        else : 
            self.model.fit(self.X, self.Y )
        if with_score:
            pred = self.predict(self.X)
            print(f"Train acc  is : {_acc(pred, self.Y)}")

    def predict(self, X):
        X = _ensure_dimensionalit(X)
        return self.model.predict(X)
